In [23]:
from openai import OpenAI

client = OpenAI()

resp = client.responses.create(
    model="o3",
    input="Write a very long novel about otters in space.",
    background=True,
)

print(resp.status)


queued


In [24]:
import time
while resp.status in {"queued", "in_progress"}:
    print(f"Current status: {resp.status}")
    time.sleep(2)
    resp = client.responses.retrieve(resp.id)

print(f"Final status: {resp.status}\nOutput:\n{resp.output_text}")


Current status: queued
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Final status: completed
Output:
Starstream Voyage: The Odyssey of the Otternauts  
A very long (but still scroll-friendly) novel

Prologue  
   A ribbon of turquoise water wound past the cedar-studded bluffs of the Pacific coast, and in that ribbon swam a pup named Ripple. She was clever even by river-otter standards, but her imagination pushed farther than any tide or current. Every night she would balance on her back, paws folded like little gloved hands, and stare past the shimmer of aurora an

In [25]:
from openai import OpenAI

client = OpenAI()

instructions = """
You are a personal math tutor. When asked a math question,
write and run code using the python tool to answer the question.
"""

resp = client.responses.create(
    model="gpt-4.1",
    tools=[
        {
            "type": "code_interpreter",
            "container": {"type": "auto", "memory_limit": "4g"},
        }
    ],
    instructions=instructions,
    input="I need to solve the equation 3x + 11 = 14. Can you help me?",
)

print(resp.output)


BadRequestError: Error code: 400 - {'error': {'message': 'Code interpreter tool cannot be used for this organization due to Zero Data Retention.', 'type': 'invalid_request_error', 'param': 'tools', 'code': 'unsupported_parameter'}}

In [26]:
from openai import OpenAI

client = OpenAI()

# For brevity, we are including file context in the example input.
# Most agentic use cases should instead equip the model with tools
# for exploring file system state.
RESPONSE_INPUT = """
The user has the following files:
<BEGIN_FILES>
===== lib/fib.py
def fib(n):
    if n <= 1:
        return n
    return fib(n-1) + fib(n-2)

===== run.py
from lib.fib import fib

def main():
  print(fib(42))
<END_FILES>

You are a helpful coding assistant that should assist the user with whatever they
ask.

User query:
Help me rename the fib() function to fibonacci()
"""

response = client.responses.create(
    model="gpt-5.1",
    input=RESPONSE_INPUT,
    tools=[{"type": "apply_patch"}],
)

# response.output may contain multiple apply_patch_call entries, e.g.:
# - update lib/fib.py
# - update run.py
patch_calls = [item for item in response.output if item.type == "apply_patch_call"]


In [27]:
print(patch_calls[0].operation)

OperationUpdateFile(diff='@@\n-def fib(n):\n+def fibonacci(n):\n@@\n-    return fib(n-1) + fib(n-2)\n+    return fibonacci(n-1) + fibonacci(n-2)\n', path='lib/fib.py', type='update_file')


In [28]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-5.1",
    instructions="The local bash shell environment is on Mac.",
    input="find me the largest pdf file in ~/Documents",
    tools=[{"type": "shell"}],
)
print(response)


Response(id='resp_045dc1e78d0b37a9016924d4965da481a38399ae0ecc85879d', created_at=1764021398.0, error=None, incomplete_details=None, instructions='The local bash shell environment is on Mac.', metadata={}, model='gpt-5.1-2025-11-13', object='response', output=[ResponseFunctionShellToolCall(id='sh_045dc1e78d0b37a9016924d4970a0481a38199555cb93a1678', action=Action(commands=["cd ~/Documents && find . -type f -name '*.pdf' -print0 | xargs -0 ls -lS | head -n 1"], max_output_length=10240, timeout_ms=None), call_id='call_HKm5nyaS8qD2KSiKvFXXzDDy', status='completed', type='shell_call', created_by=None)], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionShellTool(type='shell')], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, prompt_cache_retention=None, reasoning=Reasoning(effort='none', generate_summary=None, summary=None), safety_identifier=None, service_

In [14]:
import weave

In [15]:
weave_client = weave.init("nehal_test")


weave: Logged in as Weights & Biases user: nehalgross.
weave: View Weave data at https://wandb.ai/agent-tig/nehal_test/weave


In [17]:
import json

In [18]:
@weave.op()  # 🐝 Decorator to track requests
def extract_fruit(sentence: str) -> dict:
    client = OpenAI()
    system_prompt = (
        "Parse sentences into a JSON dict with keys: fruit, color and flavor."
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": sentence},
        ],
        temperature=0.7,
        response_format={"type": "json_object"},
    )
    extracted = response.choices[0].message.content
    return json.loads(extracted)


sentence = "There are many fruits that were found on the recently discovered planet Goocrux. There are neoskizzles that grow there, which are purple and taste like candy."
extract_fruit(sentence)


{'fruit': 'neoskizzles', 'color': 'purple', 'flavor': 'candy'}

In [21]:
%set_env OPENAI_AGENTS_DISABLE_TRACING=1


env: OPENAI_AGENTS_DISABLE_TRACING=1


In [22]:
from pydantic import BaseModel
from agents import Agent, Runner, function_tool
import agents
import weave
import asyncio



class Weather(BaseModel):
    city: str
    temperature_range: str
    conditions: str


@function_tool
def get_weather(city: str) -> Weather:
    return Weather(city=city, temperature_range="14-20C", conditions="Sunny with wind.")


agent = Agent(
    name="Hello world", instructions="You are a helpful agent.", tools=[get_weather]
)


async def main():
    result = await Runner.run(agent, input="What's the weather in Tokyo?")
    print(result.final_output)


await main()


The weather in Tokyo is currently sunny with wind, and the temperature ranges between 14°C and 20°C.
